In [1]:
import transformers
from datasets import load_dataset, list_datasets, BuilderConfig

In [31]:
import mmap
from  datasets import load_dataset
import json

dataset = load_dataset('json', data_files='test.jsonl')['train']

Found cached dataset json (C:/Users/Tran Nguyen Nhut Duy/.cache/huggingface/datasets/json/default-9729043621fc9faf/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
for i in range(10):
    print(dataset[i])

{'_id': '{"datasets_id": 0, "wiki_id": "Q1294448", "sp": 2, "sc": 0, "ep": 6, "ec": 610}', 'datasets_id': 0, 'wiki_id': 'Q1294448', 'start_paragraph': 2, 'start_character': 0, 'end_paragraph': 6, 'end_character': 610, 'article_title': 'Ági Szalóki', 'section_title': 'Life', 'passage_text': "Ági Szalóki Life She started singing as a toddler, considering Márta Sebestyén a role model. Her musical background is traditional folk music; she first won recognition for singing with Ökrös in a traditional folk style, and Besh o droM, a Balkan gypsy brass band. With these ensembles she toured around the world from the Montreal Jazz Festival, through Glastonbury Festival to the Théatre de la Ville in Paris, from New York to Beijing.\nSince 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.\nUntil now, three of her six released albums"}
{'_id': '{"datasets_id": 0, "wiki_id": "Q1294448", "sp": 6, "sc": 610, "ep": 6, "ec": 1198}'

In [38]:
def preprocess_data(examples, key='passage_text'):
        """Preprocesses the data by removing newlines, tabs, and multiple spaces
        Args:
            examples (dict): dictionary containing the data
            key (str, optional): key for the data. Defaults to 'passage_text'.
        Returns:
            dict: dictionary containing the preprocessed data
            """

        # lowercase the text
        examples[key] = examples[key].lower()
        # remove newlines
        examples[key] = examples[key].replace('\n', '')
        # remove tabs
        examples[key] = examples[key].replace('\t', '')
        # remove multiple spaces
        examples[key] = examples[key].replace('  ', ' ')
        return examples

In [34]:
print(dataset[0]['passage_text'])

Ági Szalóki Life She started singing as a toddler, considering Márta Sebestyén a role model. Her musical background is traditional folk music; she first won recognition for singing with Ökrös in a traditional folk style, and Besh o droM, a Balkan gypsy brass band. With these ensembles she toured around the world from the Montreal Jazz Festival, through Glastonbury Festival to the Théatre de la Ville in Paris, from New York to Beijing.
Since 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.
Until now, three of her six released albums


In [39]:
# preprocess the data
for i in range(len(dataset)):
    dataset[i] = preprocess_data(dataset[i])
    

TypeError: 'Dataset' object does not support item assignment

In [36]:

for i in range(10):
    print(dataset[i]['passage_text'])
    print("\n")

Ági Szalóki Life She started singing as a toddler, considering Márta Sebestyén a role model. Her musical background is traditional folk music; she first won recognition for singing with Ökrös in a traditional folk style, and Besh o droM, a Balkan gypsy brass band. With these ensembles she toured around the world from the Montreal Jazz Festival, through Glastonbury Festival to the Théatre de la Ville in Paris, from New York to Beijing.
Since 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.
Until now, three of her six released albums


were awarded Album of the Year Prize (MAHASZ Fonogram Award) in the children's records and jazz categories.
"When I sing it's not important whether it's jazz or traditional music or music of any other kind", Szalóki has said. "I feel as if I'm swimming in the music. The music is like water; or air and I fly in it."
She is trained in the Kodály method. Many of her projects are inspire

In [4]:
# combine to a passage with the same wiki_id
import collections
import tqdm
dataset_dict = collections.defaultdict(list)
for i in tqdm.tqdm(range(len(dataset))):
    dataset_dict[dataset[i]['wiki_id']].append(dataset[i]['passage_text'])
print(len(dataset_dict))

100%|██████████| 100001/100001 [00:00<00:00, 885503.34it/s]

18445


In [5]:
# connect the passages with the same wiki_id
for key in tqdm.tqdm(dataset_dict.keys()):
    dataset_dict[key] = ' '.join(dataset_dict[key])
print(len(dataset_dict))


100%|██████████| 18445/18445 [00:00<00:00, 164693.13it/s]

18445


In [25]:
for i in range(10):
    print(dataset_dict[list(dataset_dict.keys())[i]])
    print("\n")

ági szalóki life she started singing as a toddler, considering márta sebestyén a role model. her musical background is traditional folk music; she first won recognition for singing with ökrös in a traditional folk style, and besh o drom, a balkan gypsy brass band. with these ensembles she toured around the world from the montreal jazz festival, through glastonbury festival to the théatre de la ville in paris, from new york to beijing.since 2005, she began to pursue her solo career and explore various genres, such as jazz, thirties ballads, or children's songs.until now, three of her six released albums were awarded album of the year prize (mahasz fonogram award) in the children's records and jazz categories."when i sing it's not important whether it's jazz or traditional music or music of any other kind", szalóki has said. "i feel as if i'm swimming in the music. the music is like water; or air and i fly in it."she is trained in the kodály method. many of her projects are inspired by p

# model retrieval

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from models.Retrieval import WikiDataset
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# clear cuda
torch.cuda.empty_cache()

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v1')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v1')
model.eval()
model.to("cuda") if torch.cuda.is_available() else model.to("cpu")

# Create an instance of the custom WikiDataset class
dataset = WikiDataset(dataset_dict)

# Set the device (CPU or GPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Create a data loader for the dataset
batch_size = 10  # Set the desired batch size
shuffle = False  # Set to True if you want to shuffle the data
num_workers = 1  # Set the number of worker processes for data loading
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

i = 10
embd_database = []
#Tokenize sentences
for batch in data_loader:
    wiki_ids, passages = batch
    encoded_input = tokenizer(passages, padding=True, truncation=True, return_tensors='pt')
    encoded_input.to("cuda") if torch.cuda.is_available() else encoded_input.to("cpu")

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    #sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    for i in sentence_embeddings:
        print(i.shape)
    embd_database.append(sentence_embeddings)
    if i == 10:
        break
    i += 1


torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [8]:
def cosine_similarity(u, v):
    return torch.dot(u, v) / (u.norm() * v.norm())

In [9]:
question = "What was the name of the publishing company established by İnci Asena, and in which literary periodical did her poems appear?"
encoded_input = tokenizer(question, padding=True, truncation=True, return_tensors='pt')
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
question_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
#question_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
for i in range(10):
    # reshape to 1d vector
    question_embeddings = question_embeddings.reshape(-1)
    embd_database[i] = embd_database[i].reshape(-1)
    print(cosine_similarity(question_embeddings, embd_database[i]))

tensor(0.2726)
tensor(0.0226)
tensor(0.1044)
tensor(0.0008)
tensor(0.0775)
tensor(-0.0812)
tensor(0.2431)
tensor(-0.0884)
tensor(0.6726)
tensor(-0.0049)


In [31]:
print(embd_database[0].shape)

torch.Size([768])


In [21]:
print(dataset[9]['passage_text'])

been given the addition of a modern border. émile gilliéron's reconstruction of the saffron gatherer fresco has been proven to be incorrect, as it originally depicted a monkey, not a boy. in addition to restorations and reconstructions, it has been a matter of ongoing research whether the gilliérons were also involved in the forgery business producing fakes with their greek assistants. some of the artifacts attributed to the gilliérons and under suspicion of forgery include the chryselephantine snake goddesses, the ring of minos and the ring of nestor, and the well-known phaistos disc on crete. death and acknowledgements émile gilliéron


In [3]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 2.8 MB/s eta 0:00:00


In [1]:
import psycopg2

# connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="OJT_LAB_Q2",
    user="postgres",
    password="adidaphat")
# cursor = conn.cursor()
print(conn)

<connection object at 0x000001F2A0A10048; dsn: 'user=postgres password=xxx dbname=OJT_LAB_Q2 host=localhost', closed: 0>


In [34]:
import psycopg2

# connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="OJT_LAB_Q2",
    user="postgres",
    password="adidaphat")
cursor = conn.cursor()
name_table = "public.wiki_snippsets"
try:
    for i in range(10):
        cursor.execute(f"""
            INSERT INTO {name_table}
            VALUES (%s, %s, %s)""", (list(dataset_dict.keys())[i], dataset_dict[list(dataset_dict.keys())[i]], embd_database[i].reshape(-1).tolist()))
    conn.commit()
except Exception as e:
    print("Error:", e)
    conn.rollback()
cursor.close()
conn.close()

In [14]:
import numpy as np
# connect to the database
import psycopg2


conn = psycopg2.connect(
    host="localhost",
    database="OJT_LAB_Q2",
    user="postgres",
    password="adidaphat")

cursor = conn.cursor()
try:
    cursor.execute("""SELECT * FROM public.wiki_snippsets""")
    rows = cursor.fetchall()
    new_embd_database = []
    for row in rows:
        wiki_id = row[0]
        passage = row[1]
        embedding_vector = row[2]
        new_embd_database.append(embedding_vector)
    conn.commit()
except Exception as e:
    print("Error:", e)
    conn.rollback()
cursor.close()
conn.close()

In [14]:
question = "What was the name of the publishing company established by İnci Asena, and in which literary periodical did her poems appear?"
encoded_input = tokenizer(question, padding=True, truncation=True, return_tensors='pt')
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
question_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
#question_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
for i in range(10):
    # reshape to 1d vector
    question_embeddings = question_embeddings.reshape(-1)
    new_embd_database[i] = new_embd_database[i].reshape(-1)
    print(cosine_similarity(question_embeddings, new_embd_database[i]))

tensor(0.2726)
tensor(0.0226)
tensor(0.1044)
tensor(0.0008)
tensor(0.0775)
tensor(-0.0812)
tensor(0.2431)
tensor(-0.0884)
tensor(0.6726)
tensor(-0.0049)


In [33]:
import psycopg2

# connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="OJT_LAB_Q2",
    user="postgres",
    password="adidaphat")

# Call the find_nearest_passage function and print the result
cursor = conn.cursor()
question_embeddings = question_embeddings.reshape(-1)
cursor.execute("""SELECT find_similar_passage(CAST(%s AS vector(768)))""", (question_embeddings.tolist(),))
nearest_passage_text = cursor.fetchone()[0]
print("Nearest passage text:", nearest_passage_text)

# Close the cursor and connection
cursor.close()
conn.close()

Nearest passage text: i̇nci asena life she was born to muhtar and nihal in i̇stanbul, turkey in 1948. she is the younger sister of duygu asena, a notable journalist. she graduated from the school of english studies at i̇stanbul university.in 1966, she participated at the miss turkey beauty contest under the pseudonym aylin öndersev, which actually was her aunt's name, who died young. she won the contest. she married to journalist halit çapın, and gave birth to a daughter named berfu. her marriage ended with divorce lasting briefly. literature career in 1990, she established a publishing company named adam yayınları. her poems appeared in adam sanat, a literary periodical of the company. in some of her poems, she used the pen name ani toros. she also wrote books, and is a member of pen international.
